In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
   ━━━━━━━━

In [2]:
# !pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/Cosine-similarity.git
# !pip uninstall adalora_bi_eq13 -y

  Cloning https://github.com/Shannu3766/Cosine-similarity.git to /tmp/pip-req-build-shr3fo_9
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/Cosine-similarity.git /tmp/pip-req-build-shr3fo_9
  Resolved https://github.com/Shannu3766/Cosine-similarity.git to commit 76574c3690a5ecf452ca0e2bfb25b34b38b6bec5
  Preparing metadata (setup.py) ... done
  Created wheel for adalora_bi_eq13: filename=adalora_bi_eq13-0.1.0-py3-none-any.whl size=10519 sha256=be107333baed7b79b084143de5ab183985d035b294e41ea277b475498d7450f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-gyl2cpm9/wheels/76/bb/9e/d30494391cfeaff12e87f494af145e11c56eaec39eff965930
Successfully built adalora_bi_eq13


In [3]:
import os
from typing import Optional

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
from adaptive_lora_gradient.callbacks import AdaptiveLoRACallback

2025-12-06 09:14:00.101476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765012440.498055      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765012440.611131      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" 
output_dir = "./tinyllama-qnli-lora"
seed = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [5]:
batch_size = 16
eval_batch_size = 32
num_train_epochs = 5
max_length = 128
learning_rate = 3e-5
weight_decay = 0.01
rank=8
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=rank,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ],)

In [6]:

def clean_text(s: Optional[str]) -> str:
    if s is None:
        return ""
    return " ".join(str(s).strip().split())

def build_one_shot_demo(example: dict) -> str:
    q = clean_text(example["question"])
    c = clean_text(example["context"])
    lbl = example.get("label_text", "Yes")
    return f"Example:\nQuestion: {q}\nContext: {c}\nAnswer (Yes/No): {lbl}\n\n"

In [7]:
ONE_SHOT_EXAMPLE = {
    "question": "Who wrote Hamlet?",
    "context": "Hamlet was written by William Shakespeare and first performed in the early 17th century.",
    "label_text": "Yes",  # "Yes" => the context contains the answer
}

from datasets import load_dataset
import time

def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):
    """
    Repeatedly tries to load a dataset until it succeeds.
    - max_retries: None → infinite retries
    - wait: base wait time (exponential backoff)
    """
    attempt = 0

    while True:
        try:
            if name is not None:
                ds = load_dataset(path, name)
            else:
                ds = load_dataset(path)

            print(f"Dataset loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  # exponential cap

            print(f"[Attempt {attempt}] Failed to load dataset: {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            # If user gives a max_retries limit
            if max_retries is not None and attempt >= max_retries:
                print("Max retries reached. Raising error.")
                raise e

            time.sleep(wait_time)
dataset = load_dataset_with_retry("glue", "qnli")
# dataset = load_dataset("glue", "qnli")

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Dataset loaded successfully after 0 attempts.


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

if tokenizer.pad_token is None:
    # safe default: use eos_token as pad
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [9]:

# Load tokenizer


def preprocess_function(examples, *, max_length: int = max_length, add_demo: bool = False):
    """
    Build instruction-like prompts and tokenize them.
    - add_demo: if True, prepends a 1-shot example to each prompt (can help decoder-only models)
    """
    questions = [clean_text(q) for q in examples["question"]]
    contexts = [clean_text(s) for s in examples["sentence"]]

    demo_str = build_one_shot_demo(ONE_SHOT_EXAMPLE) if add_demo else ""

    prompts = []
    for q, c in zip(questions, contexts):
        # Instruction-style prompt ending with a short, constrained label target
        prompt = (
            "You are a helpful assistant.\n"
            f"{demo_str}"
            f"Question: {q}\n"
            f"Context: {c}\n"
            "Answer (Yes/No):"
        )
        prompts.append(prompt)

    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=max_length,
        padding=False, 
    )

    if "label" in examples:
        tokenized["labels"] = examples["label"]
    elif "labels" in examples:
        tokenized["labels"] = examples["labels"]

    return tokenized

print("Tokenizing dataset...")
tokenized = dataset.map(
    lambda ex: preprocess_function(ex, max_length=max_length, add_demo=False),
    batched=True,
    remove_columns=["question", "sentence", "idx"],
)

if "label" in tokenized["train"].column_names and "labels" not in tokenized["train"].column_names:
    tokenized = tokenized.rename_column("label", "labels")

tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest", pad_to_multiple_of=8)
train_dataset = tokenized["train"].select(range(10000))
eval_dataset = tokenized["validation"].select(range(1500))

Tokenizing dataset...


Map:   0%|          | 0/104743 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

In [10]:
print("Loading model...")

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

print("Model Loaded")

Loading model...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Loaded


In [11]:
print("Model Loaded")
# Resize token embeddings if tokenizer changed
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

# -------------------------
# Apply LoRA (PEFT)
# -------------------------
print("Applying LoRA (PEFT)...")
model = get_peft_model(model, peft_config)

Model Loaded
Applying LoRA (PEFT)...


In [12]:
val_dataloader = torch.utils.data.DataLoader(
    eval_dataset,
    batch_size=eval_batch_size,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=torch.cuda.is_available(),
    num_workers=2,
)

In [13]:
from peft.tuners.lora import LoraLayer

def get_lora_module_names(peft_model):
    names = []
    for name, module in peft_model.named_modules():
        if isinstance(module, LoraLayer):
            names.append(name)
    return names

lora_names = get_lora_module_names(model)   # `model` is your PEFT-wrapped model
print(f"🔢 Number of LoRA modules: {len(lora_names)}\n")

# for n in lora_names:
    # print(" ", n)
num_lora_modules = len(lora_names)
TOTAL_RANK_BUDGET = rank * num_lora_modules

avg_rank_per_module = TOTAL_RANK_BUDGET / num_lora_modules
print(
    f"💡 With TOTAL_RANK_BUDGET={TOTAL_RANK_BUDGET} over "
    f"{num_lora_modules} modules, avg rank ≈ {avg_rank_per_module:.2f}"
)


🔢 Number of LoRA modules: 154

💡 With TOTAL_RANK_BUDGET=1232 over 154 modules, avg rank ≈ 8.00


In [14]:
adaptive_callback = AdaptiveLoRACallback(
    total_rank=TOTAL_RANK_BUDGET,
    val_dataloader=val_dataloader,
    score_smoothing_beta=0.8,   
    update_interval=2,        
    warmup_epochs=1,    
    cooldown_epochs=1,  
    min_rank=4,
    tau=0.9
)

print("✅ Adaptive LoRA callback ready.")

✅ Adaptive LoRA callback ready.


In [15]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    return {"accuracy": acc["accuracy"]}

# -------------------------
# TrainingArguments + Trainer
# -------------------------
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    num_train_epochs=num_train_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=2,
    metric_for_best_model="accuracy",
    seed=seed,
    fp16=False,
    bf16=True,
    push_to_hub=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback]
)

print("Starting training...")
trainer.train()

print("Saving model and adapters...")
trainer.save_model(output_dir)
print("Saved model to", output_dir)

/tmp/ipykernel_20/3391662561.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...
⏳ AdaptiveLoRA: Warmup (Epoch 1). Skipping update.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.370100,0.344934,0.856000
2,0.298500,0.385924,0.857333
3,0.218500,0.337491,0.883333
4,0.205900,0.349047,0.884667
5,0.156100,0.356275,0.883333


⏭️ AdaptiveLoRA: Interval skip (Epoch 2). Keeping ranks.

--- AdaptiveLoRA: Adapting Ranks for Epoch 3 ---


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Computing Importance:   0%|          | 0/188 [00:00<?, ?it/s]

Rank Updates (Epoch 3):
  - base_model.model.model.layers.0.self_attn.q_proj: r=8 → 7 (Score: 0.1869)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 0.3201)
  - base_model.model.model.layers.0.self_attn.v_proj: r=8 → 7 (Score: 0.2254)
  - base_model.model.model.layers.0.self_attn.o_proj: r=8 → 9 (Score: 0.4896)
  - base_model.model.model.layers.0.mlp.gate_proj: r=8 (Unchanged, Score: 0.3672)
  - base_model.model.model.layers.0.mlp.up_proj: r=8 (Unchanged, Score: 0.3791)
  - base_model.model.model.layers.0.mlp.down_proj: r=8 (Unchanged, Score: 0.3790)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 0.2914)
  - base_model.model.model.layers.1.self_attn.k_proj: r=8 → 12 (Score: 0.6755)
  - base_model.model.model.layers.1.self_attn.v_proj: r=8 (Unchanged, Score: 0.3775)
  - base_model.model.model.layers.1.self_attn.o_proj: r=8 (Unchanged, Score: 0.3613)
  - base_model.model.model.layers.1.mlp.gate_proj: r=8 (Unchanged, Score: 0.326

In [16]:
results=trainer.evaluate()
print(results)

{'eval_loss': 0.3562753200531006, 'eval_accuracy': 0.8833333333333333, 'eval_runtime': 177.0698, 'eval_samples_per_second': 8.471, 'eval_steps_per_second': 0.265, 'epoch': 5.0}
